# 5단계: 배포 파이프라인 추가

<div class="alert alert-warning"> 이 노트북은 <code>SageMaker Distribution Image 3.6.1</code>을 사용하는 SageMaker Studio JupyterLab 인스턴스에서 SageMaker Python SDK 버전 <code>2.255.0</code>으로 마지막 테스트되었습니다</div>

이전 단계에서 자동화된 데이터 처리 및 모델 빌드 파이프라인을 구현했습니다. 파이프라인의 각 실행은 새로운 버전의 모델을 생성합니다. 이 노트북은 ML 워크플로우에서 자동화된 모델 배포 단계를 구현합니다.

[SageMaker MLOps 프로젝트 템플릿](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-templates-sm.html#sagemaker-projects-templates-git-code-pipeline)의 두 번째 부분을 사용하여 바로 사용할 수 있는 모델 배포 CI/CD 파이프라인을 생성합니다.

MLOps 프로젝트 템플릿은 SageMaker 모델 레지스트리의 모델을 실시간 추론을 위한 SageMaker 엔드포인트에 배포하기 위한 CodePipeline 파이프라인을 구현합니다. 새 모델 버전이 모델 레지스트리에 등록되고 승인되면 파이프라인이 자동으로 시작되어 배포를 수행합니다.

||||
|---|---|---|
|1. |노트북에서 실험 ||
|2. |SageMaker AI 처리 작업 및 SageMaker SDK로 확장 ||
|3. |ML 파이프라인, 모델 레지스트리 및 피처 스토어로 운영화 ||
|4. |모델 빌드 CI/CD 파이프라인 추가 ||
|5. |모델 배포 파이프라인 추가 |**<<<< 현재 위치**|
|6. |모델 및 데이터 모니터링 추가 ||

<div class="alert alert-info"> 이 노트북에서 JupyterLab의 <code>Python 3</code> 커널을 사용하고 있는지 확인하세요.</div>

In [ ]:
import boto3
import sagemaker 
from time import gmtime, strftime, sleep
from IPython.display import HTML

sagemaker.__version__

In [ ]:
%store -r 

%store

try:
    initialized
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN 00-start-here notebook   ")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")

In [ ]:
sm = boto3.client("sagemaker")

In [ ]:
try:
    print(project_name)
    print(project_id)
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("You must complete the notebook 04-sagemaker-project")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++")

## Studio UI에서 프로젝트 탐색

<div class="alert alert-info">4단계 노트북을 실행하고 MLOps 프로젝트를 성공적으로 프로비저닝하며 프로젝트 파이프라인을 최소 한 번 실행했는지 확인하세요.</div>

다음 코드 셀에서 구성된 링크를 클릭하여 Studio UI에서 프로젝트와 모델 패키지를 확인하세요. 모델 패키지 그룹에 등록된 모델의 버전이 최소 하나는 있어야 합니다.

In [ ]:
# Show the project link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/projects/{}/">the project</a> in the Studio UI</b>'.format(
            domain_id, region, project_name))
)

# Show the model package link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/models/registered-models/{}-{}/versions">the model package versions</a> in the Studio UI</b>'.format(
            domain_id, region, project_name, project_id))
)

In [ ]:
# check that the project exists
project_data = sm.describe_project(ProjectName=project_name)

In [ ]:
assert project_data['ProjectStatus'] == 'CreateCompleted', 'Project must be created at this point!'

In [ ]:
# check that at least one model version is registered in the model registry
model_packages = sm.list_model_packages(
    ModelPackageGroupName=f'{project_name}-{project_id}',
    ModelApprovalStatus='PendingManualApproval')

In [ ]:
assert len(model_packages['ModelPackageSummaryList']) > 0, 'You must have at least one model version in the status PendingManualApproval'

---

## 모델 배포를 위한 MLOps 프로젝트 작업
템플릿은 모델 배포 단계를 지정하는 구성 파일, 엔드포인트를 인프라로 정의하는 AWS CloudFormation 템플릿, 엔드포인트 테스트를 위한 시드 코드가 포함된 GitHub 저장소를 프로비저닝합니다.

이 템플릿은 다음 리소스를 제공합니다:

1. 스테이징 및 프로덕션 환경의 엔드포인트에 모델을 배포하는 시드 코드가 포함된 GitHub 저장소
2. `source`, `build`, `deploy-to-staging`, `deploy-to-production` 단계가 있는 AWS CodePipeline 파이프라인. `source` 단계는 GitHub 저장소를 가리키고, `build` 단계는 해당 저장소에서 코드를 가져와 배포할 CloudFormation 스택을 생성합니다. `deploy-to-staging` 및 `deploy-to-production` 단계는 CloudFormation 스택을 각각의 환경에 배포합니다. 스테이징과 프로덕션 빌드 단계 사이에 수동 승인 단계가 있어 MLOps 엔지니어가 모델을 프로덕션에 배포하기 전에 승인해야 합니다.
3. 모델 패키지 버전이 승인되거나 거부될 때 CodePipeline 파이프라인 실행을 시작하는 Amazon EventBridge 규칙
4. 플레이스홀더 단위 테스트 후에도 수동 승인 단계가 있습니다. 플레이스홀더 테스트를 대체할 자체 테스트를 구현할 수 있습니다.

프로젝트 템플릿은 또한 CodePipeline 및 CodeBuild 아티팩트와 SageMaker 파이프라인 실행에서 생성된 모든 아티팩트를 포함한 아티팩트를 저장하기 위한 Amazon S3 버킷을 배포합니다.

다음 다이어그램은 아키텍처를 보여줍니다.

<img src="img/mlops-model-deploy.png" width="600"/>

프로젝트에 대한 구성 변경을 구현할 필요가 없습니다. 모델 배포 파이프라인은 즉시 작동합니다.
모델 배포 파이프라인을 시작하려면 모델 레지스트리에서 모델 버전을 승인해야 합니다.

### 모델 버전 승인
모델 버전을 승인하면 MLOps 프로젝트가 모델 배포 프로세스를 시작합니다.

첫 번째 단계에서 모델 배포 파이프라인은 모델 버전을 스테이징 SageMaker 실시간 추론 엔드포인트에 배포합니다.

Studio의 모델 레지스트리에서 모델 버전을 승인하거나 노트북에서 프로그래밍 방식으로 수행할 수 있습니다. 프로그래밍 방식으로 해보겠습니다.

In [ ]:
# list all model packages and select the latest one
model_packages = [
    p["ModelPackageSummaryList"] for p in
    sm.get_paginator('list_model_packages').paginate(
        ModelPackageGroupName=f'{project_name}-{project_id}',
        ModelApprovalStatus='PendingManualApproval',
        SortBy="CreationTime",
        SortOrder="Descending",
    )
][0]

In [ ]:
if len(model_packages) == 0:
    raise Exception(f"No model package is found for {model_package_group_name} model package group. Run a model creation pipeline first.")

print(f"There are {len(model_packages)} model versions in the {model_package_group_name} model package group")
print(f"Approve the most recent model package:")

latest_model_package_arn = model_packages[0]["ModelPackageArn"]
print(latest_model_package_arn)

다음 문은 모델 레지스트리에서 가장 최근 모델 패키지의 `ModelApprovalStatus`를 `Approved`로 설정합니다. 모델 패키지 상태 변경은 EventBridge 규칙을 시작하고, 이 규칙은 모델 배포와 함께 CodePipeline CI/CD 파이프라인을 시작합니다

In [ ]:
model_package_update_response = sm.update_model_package(
    ModelPackageArn=latest_model_package_arn,
    ModelApprovalStatus="Approved",
)

다음 코드 셀에서 구성된 링크를 클릭하여 Studio UI의 모델 레지스트리에서 마지막 모델 버전의 **상태**가 `Approved`로 변경된 것을 확인하세요:

In [ ]:
# Show the model package version link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/models/registered-models/{}-{}/versions/version-{}/overview">the model package version</a> in the Studio UI</b>'.format(
            domain_id, region, project_name, project_id, latest_model_package_arn.split('/')[-1]))
)

### 배포 파이프라인 실행
위 코드 셀에서 모델 버전이 승인되면 모델 배포 CI/CD 파이프라인은 다음 작업을 수행합니다:
1. SageMaker 엔드포인트용 CloudFormation 템플릿의 스테이징 및 프로덕션 매개변수가 포함된 CloudFormation 매개변수 구성 파일 생성
1. `<PROJECT-NAME>-staging` 이름으로 SageMaker 실시간 추론 엔드포인트 생성
1. 스테이징 엔드포인트에서 테스트 스크립트 실행
1. [AWS CodePipeline 콘솔](https://console.aws.amazon.com/codesuite/codepipeline)에서 테스트 결과가 수동으로 승인될 때까지 대기
1. 현재 계정에서 `<PROJECT-NAME>-prod` 이름으로 SageMaker 엔드포인트 생성

파이프라인이 스테이징 엔드포인트 배포를 완료할 때까지 약 5-10분 기다리세요. Studio UI의 **Deployments** > **Endpoints**에서 엔드포인트 상태를 확인할 수 있습니다:

In [ ]:
# Show the inference endpoints link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/inference-experience/endpoints">inference endpoints</a> in the Studio UI</b>'.format(
            domain_id, region))
)

엔드포인트 상태가 `Creating`에서 `InService`로 변경된 후 스테이징 엔드포인트가 완전히 작동합니다. CodePipeline 파이프라인의 **DeployStaging** 단계를 수동으로 승인하여 프로덕션 단계로 모델 배포 프로세스를 시작할 수 있습니다. 다음 섹션에서 모델 배포를 승인하고 프로덕션 엔드포인트로의 배포 두 번째 단계를 시작합니다.

### 프로덕션에 모델 버전 배포

<div style="border: 4px solid coral; text-align: center; margin: auto;">
    <p style=" text-align: center; margin: auto;">스테이징 엔드포인트 상태가 InService로 변경될 때까지 기다린 후 다음 코드 셀을 계속 진행하세요.
    </p>
</div>

In [ ]:
while sm.describe_endpoint(EndpointName=f'{project_name}-staging')['EndpointStatus'] != 'InService':
    sleep(10)
    print(f'Waiting untile the endpoint {project_name}-staging is operational')

print(f'Endpoint {project_name}-staging is InService')

다음 코드 셀은 CodePipeline 승인 링크를 구성합니다 - 링크를 클릭하여 파이프라인 실행 워크플로우가 있는 CodePipeline 콘솔을 여세요.

**DeployStaging 단계**에서 **ApproveDeployment** 단계의 **Review**를 선택하세요. `TestStaging` 단계가 `Succeeded` 상태로 완료될 때까지 기다려야 할 수 있습니다.

<img src="img/deploy-staging-review.png" alt="approve deployment" width="800px">

**Review** 대화 상자에서 **Approve**를 선택하고 **Submit**을 선택하세요.

In [ ]:
# Show the approval link
display(
    HTML(
        '<b>Please approve the manual step in <a target="top" href="https://console.aws.amazon.com/codesuite/codepipeline/pipelines/sagemaker-{}-{}-modeldeploy/view?region={}">AWS CodePipeline</a></b>'.format(
            project_name, project_id, region)
    )
)

**DeployStaging** 단계를 승인하면 배포 파이프라인이 계속되어 모델을 프로덕션 엔드포인트에 배포합니다. 엔드포인트를 보려면 Studio UI에서 **Deployments** > **Endpoints**를 선택하거나 아래 링크를 클릭하세요.

In [ ]:
# Show the inference endpoints link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/inference-experience/endpoints">inference endpoints</a> in the Studio UI</b>'.format(
            domain_id, region))
)

CI/CD 배포 파이프라인이 계속되면서 이전에 배포된 스테이징 엔드포인트가 `InService` 상태에 있는 것과 함께 프로덕션 엔드포인트가 `Creating` 상태에 있는 것을 볼 수 있습니다.

5-10분 후 배포가 완료되고 두 엔드포인트 모두 `InService` 상태가 됩니다.

다음 셀에서 구성된 링크를 클릭하여 프로젝트에 속한 엔드포인트를 확인하세요.
프로젝트와 엔드포인트가 메타데이터 및 리소스 태그를 통해 연결되어 있기 때문에 `staging`과 `prod` 두 엔드포인트 모두 배포 프로젝트에서 볼 수 있습니다.

In [ ]:
# Show the project link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/projects/{}/endpoints">endpoints of the project {}</a> in the Studio UI</b>'.format(
            domain_id, region, project_name, project_name))
)

## 요약
이 노트북에서는 다음 기능을 가진 자동화된 CI/CD 배포 파이프라인을 구현했습니다:
- SageMaker 실시간 추론 엔드포인트 배포를 위한 CloudFormation 템플릿 사용
- 모델 레지스트리에서 모델 승인이 모델 배포 파이프라인을 시작
- 모델 배포 파이프라인은 스테이징과 프로덕션 두 단계를 포함하며, 스테이징 엔드포인트에 대한 자동화된 테스트와 프로덕션 배포에 대한 수동 승인, 그리고 프로덕션 엔드포인트의 최종 배포를 포함합니다.

---

## 정리
<div style="border: 4px solid coral; text-align: center; margin: auto;">
    <p style=" text-align: center; margin: auto;">
    6단계 노트북(데이터 및 모델 품질 모니터링)을 실행할 예정이라면 엔드포인트 중 최소 하나는 유지해야 합니다. 여기서 워크샵을 마치고 6단계 노트북을 실행하지 않는다면 <b>정리 노트북(99-clean-up.ipynb)</b>으로 이동하여 정리 지침에 따라 AWS 계정에서 요금이 발생하지 않도록 하세요.
    <br>
    <br>
    AWS에서 제공하는 AWS 계정을 사용하는 경우 정리를 실행할 필요가 없습니다.
    </p>
</div>

## 6단계 계속하기
6단계 [노트북](06-monitoring.ipynb)을 여세요.

## 실제 프로젝트를 위한 추가 개발 아이디어
- AWS KMS 키를 사용한 종단 간 데이터 암호화 추가
- 특정 프로젝트 요구사항을 충족하는 모델 배포를 위한 [사용자 정의 SageMaker 프로젝트 템플릿](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-templates-custom.html) 생성
- ML 워크플로에 [다중 계정 모델 배포](https://aws.amazon.com/blogs/machine-learning/multi-account-model-deployment-with-amazon-sagemaker-pipelines/) 추가
- CodePipeline 파이프라인의 플레이스홀더에 자동화된 모델 테스트 추가
- [Amazon SageMaker Inference Recommender](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-recommender.html)를 사용하여 추론 엔드포인트에 대한 자동화된 로드 테스트 실행 및 최적의 인스턴스 유형과 구성 선택

## 추가 리소스
- [머신러닝 모델을 실시간 추론 엔드포인트에 배포하기](https://aws.amazon.com/getting-started/hands-on/machine-learning-tutorial-deploy-model-to-real-time-inference-endpoint/)
- [SageMaker MLOps 프로젝트 연습](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-walkthrough.html)
- [SageMaker Immersion Day의 Amazon SageMaker Pipelines 실습](https://catalog.us-east-1.prod.workshops.aws/workshops/63069e26-921c-4ce1-9cc7-dd882ff62575/en-US/lab6)
- [Amazon SageMaker 보안 MLOps](https://github.com/aws-samples/amazon-sagemaker-secure-mlops)
- [Amazon SageMaker ML 모델을 위한 테스트 접근법](https://aws.amazon.com/blogs/machine-learning/testing-approaches-for-amazon-sagemaker-ml-models/)
- [Amazon SageMaker의 모델 호스팅 패턴 블로그 시리즈](https://aws.amazon.com/blogs/machine-learning/model-hosting-patterns-in-amazon-sagemaker-part-1-common-design-patterns-for-building-ml-applications-on-amazon-sagemaker/)
- [Amazon SageMaker 배포 가드레일을 사용한 고급 배포 전략 활용하기](https://aws.amazon.com/blogs/machine-learning/take-advantage-of-advanced-deployment-strategies-using-amazon-sagemaker-deployment-guardrails/)
- [Amazon SageMaker를 사용한 실시간 추론 모델 서빙 엔드포인트의 MLOps 배포 모범 사례](https://aws.amazon.com/blogs/machine-learning/mlops-deployment-best-practices-for-real-time-inference-model-serving-endpoints-with-amazon-sagemaker/)

# kernel 정지

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>